<a href="https://colab.research.google.com/github/Arsyad1999/Arsyad1999/blob/main/Chatbot_basic_Conversation_in_Japanese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import librarinya
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Pake kamus buat mewakili intents file JSON 
data = {"intents": [
             {"tag": "salam",
              "patterns": ["こんにちは", "お元気ですか?", "何してるの？", "おはよう", "こんばんは?"],
              "responses": ["こんにちは ", "もちろん元気です", "何もしない", "おはようございます！", "こんばんは?"],
             },
             {"tag": "umur",
              "patterns": ["何歳ですか?", "あなたの誕生日はいつですか?", "いつ生まれですか?"],
              "responses": ["21歳です", "誕生日は11月2日です", "1999年に生まれました", "1999/11/02"]
             },
             {"tag": "kegiatan",
              "patterns": ["何している?","毎朝何時に起きますか?", "毎日何をしてますか？"],
              "responses": ["何もしてない", "五時ごろにおきます", "毎日大学で勉強している"]
             },
             {"tag": "nama",
              "patterns": ["お名前は何ですか?", "あなたは誰ですか?", "あなた誰?"],
              "responses": ["名前は田中です", "田中と申します", "田中です"]
             },
             {"tag": "penutup",
              "patterns": [ "じゃあ", "今行きます", "バイ", "またね", "また会いましょう"],
              "responses": ["あなたと会えて良かった", "またね", "明日話しましょう"]
             }
]}

In [ ]:
# Menginisialiasi lemmatizer guna mendapatkan suku kata
lemmatizer = WordNetLemmatizer()
# tiap list yang dibuat
words = []
classes = []
doc_X = []
doc_y = []
# Loop melalui semua intens
# tokenize setiap pola dan tambahkan token ke kata, pola dan
# tag terkait ke list terkaitnya
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # tambahkan tag ke classes jika belum ada 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize semua kata dalam kosakata dan ubah mereka ke lowercase 
# Jika kata tidak muncul dalam tanda baca
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# Mensortir kosakata dan class dalam urutan abjad dan mengambil #set untuk memastikan tidak terjasi duplikasi
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
#cek 1 per 1


In [ ]:
print(words)

['あなたの誕生日はいつですか', 'あなたは誰ですか', 'あなた誰', 'いつ生まれですか', 'おはよう', 'お元気ですか', 'お名前は何ですか', 'こんにちは', 'こんばんは', 'じゃあ', 'またね', 'また会いましょう', 'バイ', '今行きます', '何している', '何してるの？', '何歳ですか', '毎日何をしてますか？', '毎朝何時に起きますか']


In [ ]:
print(classes)

['kegiatan', 'nama', 'penutup', 'salam', 'umur']


In [ ]:
print(doc_X)

['こんにちは', 'お元気ですか?', '何してるの？', 'おはよう', 'こんばんは?', '何歳ですか?', 'あなたの誕生日はいつですか?', 'いつ生まれですか?', '何している?', '毎朝何時に起きますか?', '毎日何をしてますか？', 'お名前は何ですか?', 'あなたは誰ですか?', 'あなた誰?', 'じゃあ', '今行きます', 'バイ', 'またね', 'また会いましょう']


In [ ]:
print(doc_y)

['salam', 'salam', 'salam', 'salam', 'salam', 'umur', 'umur', 'umur', 'kegiatan', 'kegiatan', 'kegiatan', 'nama', 'nama', 'nama', 'penutup', 'penutup', 'penutup', 'penutup', 'penutup']


In [ ]:
# list untuk training data
training = []
out_empty = [0] * len(classes)
# membuat model bags of words
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # tandai index class yang terkait dengan pola saat ini
    # ke
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # tambahkan 1 hot encoded bow dan class terkait ke training data
    training.append([bow, output_row])
# men shuffle data dan ubah menjadi Array
random.shuffle(training)
training = np.array(training, dtype=object)
# pisah the features dan label target 
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [ ]:
# Menentukan beberapa Parameter
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# Model Deep Learning
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               2560      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 11,141
Trainable params: 11,141
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [==============================] - 1s 698ms/step - loss: 1.6343 - accuracy: 0.2105
Epoch 2/200
1/

In [ ]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
#Uji Coba Chatbotnya
# running the chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

こんばんは?
またね
またね
